# Constitutive Models

In [ ]:
import pyvista as pv

pv.set_jupyter_backend("static")

%load_ext autoreload
%autoreload 2

In [ ]:
import logging

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from materialite import (
    Material,
    Sphere,
    Order2SymmetricTensor,
    Order4SymmetricTensor,
    Orientation,
    SlipSystem,
)
from materialite.models.small_strain_fft import (
    SmallStrainFFT,
    LoadSchedule,
    Elastic,
    ElasticViscoplastic,
    IsotropicElasticPlastic,
    voce,
    linear,
)

**Example 1: Uniaxial tension with linear elasticity**

First, create a `Material` with an "orientation" field set to the identity everywhere. `SmallStrainFFT` requires information about the orientation of each point, although this is only relevant for anisotropic constitutive laws.

In [ ]:
material = Material(dimensions=[8, 8, 8]).create_uniform_field(
    "orientation", Orientation.identity()
)

`SmallStrainFFT` requires a constitutive model and the boundary conditions as inputs. Here, we define a simple linear elastic model. The only input is the stiffness tensor. We will use units of MPa throughout.

The boundary conditions come from a `LoadSchedule` object. Here, we define uniaxial tension in the `z` direction with a strain rate of $1 /\mathrm{s}$ (i.e., $\dot{E}_{33} = 1/\mathrm{s}$, with all average stress components other than $\Sigma_{33}$ equal to zero)

In [ ]:
stiffness_tensor = Order4SymmetricTensor.from_isotropic_constants(
    modulus=200000, shear_modulus=77000
)
constitutive_model = Elastic(stiffness=stiffness_tensor)
load_schedule = LoadSchedule.from_constant_uniaxial_strain_rate(
    magnitude=1.0, direction="z"
)

Now we can instantiate and run the model. `SmallStrainFFT` takes the following inputs:
- The `LoadSchedule` object we created.
- The end time of the simulation.
- The initial time increment.
- The constitutive model we defined.

We will take one time increment of $0.001$ s.

In [ ]:
model = SmallStrainFFT(
    load_schedule=load_schedule,
    end_time=0.001,
    initial_time_increment=0.001,
    constitutive_model=constitutive_model,
)
material = material.run(model)

Running the model adds stress and strain field to the `Material`. The stress has one nonzero component, which is the same everywhere and simply equal to the total applied strain multiplied by Young's modulus.

In [ ]:
material.extract("stress").mean().stress_voigt

In [ ]:
material.plot("stress", component=2, color_lims=(190, 210))

**Example 2: Uniaxial tension with isotropic plasticity**

This is very similar to the elastic simulation. We define a `Material`, an isotropic elastic-plastic constitutive model, and the same `LoadSchedule` as before. The new constitutive model requires:
- Young's modulus
- Shear modulus
- Initial yield stress
- Hardening law. Here, we use linear hardening.
- Hardening properties

In [ ]:
material = Material(dimensions=[8, 8, 8]).create_uniform_field(
    "orientation", Orientation.identity()
)
stiffness_tensor = Order4SymmetricTensor.from_isotropic_constants(
    modulus=200000, shear_modulus=77000
)
constitutive_model = IsotropicElasticPlastic(
    modulus=200000,
    shear_modulus=77000,
    yield_stress=300,
    hardening_function=linear,
    hardening_properties={"hardening_rate": 1000},
)
load_schedule = LoadSchedule.from_constant_uniaxial_strain_rate(
    magnitude=1.0, direction="z"
)

Running the model is similar to before. Now the total time of the simulation will be $0.01$ s, corresponding to $1\%$ strain. We will also request stress and strain output at several times during the simulation so that we can construct the stress-strain curve.

In [ ]:
model = SmallStrainFFT(
    load_schedule=load_schedule,
    end_time=0.01,
    initial_time_increment=0.001,
    constitutive_model=constitutive_model,
)
num_output_times = 10
output_times = np.linspace(start=0.001, stop=0.01, num=num_output_times)
material = material.run(model, output_times=output_times)

Now the "stress" and "strain" fields in the `Material` contain the pointwise stresses and strains across all the requested output times. Each will be an `Order2SymmetricTensor` with dimensions "p" (for each point in space) and "t" (for each time step). See the Tensor Demo for details. Here, we'll verify the dimensions and print the stress at the end of the simulation.

In [ ]:
stress = material.extract("stress")
print(f"Stress dimensions: {stress.dims}")
print(stress[:, -1])

Extract the mean stress and strain in the material in the loading direction for each time step.

In [ ]:
stress = material.extract("stress").mean("p").components[:, 2]
strain = material.extract("strain").mean("p").components[:, 2]

Plot the mean stress-strain curve. Add a point at (0,0) for visualization.

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.insert(strain, 0, 0), np.insert(stress, 0, 0))
ax.set_xlabel("Axial strain [-]")
ax.set_ylabel("Axial stress [MPa]")

Plot the axial stress (normal stress in the loading direction) at each point. `material.plot()` automatically plots the value at the last time step/end of the simulation.

In [ ]:
material.plot("stress", component=2, color_lims=(290, 310))

**Example 3: Uniaxial tension of a single-phase FCC material**

First, create a `Material` with a periodic Voronoi tesselation of grains, each of which has a random orientation. Visualize the grain structure.

In [ ]:
num_grains = 30
grains_list = np.arange(num_grains)
rng = np.random.default_rng(1)

material = (
    Material(dimensions=[16, 16, 16])
    .create_voronoi(num_grains, periodic=True, label="grain", rng=rng)
    .assign_random_orientations(
        region_label="grain", orientation_label="orientation", rng=rng
    )
)
material.plot("grain")

Next, we define a crystalline elastic-viscoplastic constitutive model with a Voce hardening law for the material.
- Stiffness tensor, $\mathbb{C}$
- Slip systems operative in the material. We assume an FCC material here, so the octahedral slip systems are active.
- Reference slip rate, $\dot{\gamma}_0$
- Rate exponent, $m$
- Initial slip resistance, $\tau_0$
- Voce hardening law parameters: $\tau_1$, $\theta_0$, $\theta_1$

In [ ]:
stiffness_tensor = Order4SymmetricTensor.from_cubic_constants(
    C11=243300, C12=156700, C44=117800
)
constitutive_model = ElasticViscoplastic(
    stiffness=stiffness_tensor,
    slip_systems=SlipSystem.octahedral(),
    reference_slip_rate=1.0,
    rate_exponent=10.0,
    slip_resistance=143.0,
    hardening_function=voce,
    hardening_properties={"tau_1": 50.0, "theta_0": 1450.0, "theta_1": 95.0},
)

Here, we'll still apply a constant uniaxial strain rate, but using a more general `LoadSchedule` constructor, `LoadSchedule.from_constant_rates()`, as a demonstration. The inputs are:
* `strain_rate`: constant mean strain rate during the simulation; must be an `Order2SymmetricTensor`.
* `stress_rate`: constant mean stress rate during the simulation; must be an `Order2SymmetricTensor`.
* `stress_mask`: numpy array with six values, where each value is a `1` or `0`. Indices correspond to the Voigt components of the `stress_rate`. `1` indicates that the corresponding mean stress component in `stress_rate` is enforced in the simulation. `0` indicates that the corresponding mean strain rate is enforced in the simulation.

In [ ]:
strain_rate = Order2SymmetricTensor.from_strain_voigt([0, 0, 1.0, 0, 0, 0])
stress_rate = Order2SymmetricTensor.zero()
stress_mask = np.array([1, 1, 0, 1, 1, 1])
load_schedule = LoadSchedule.from_constant_rates(
    strain_rate=strain_rate, stress_rate=stress_rate, stress_mask=stress_mask
)

The constitutive models can have some internal state variables that we may be interested in. Each one comes with an `available_state_variables` attribute that you can query.

In [ ]:
constitutive_model.available_state_variables

Creating and running the `SmallStrainFFT` model proceeds as before. Here, we will request slip system shear strain outputs using the `output_variables` keyword argument, which must be a list. We will also use Python's `logging` module to get information during the simulation.  `INFO` marks the start of new increments. `DEBUG` provides additional debugging information related to convergence.

NOTE: The syntax of the `logging` module can be tricky. We suggest calling `logging.basicConfig()` immediately after importing `logging` to set up the logger. Then you can choose the outputs you want by calling `logger = logging.getLogger()` followed by `logger.setLevel(...)`.

In [ ]:
model = SmallStrainFFT(
    load_schedule=load_schedule,
    end_time=0.007,
    initial_time_increment=0.001,
    constitutive_model=constitutive_model,
)
num_output_times = 7
output_times = np.linspace(start=0.001, stop=0.007, num=num_output_times)

# Set up logger
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

material = material.run(
    model, output_times=output_times, output_variables=["slip_system_shear_strains"]
)

# Reset logger to WARNING to avoid INFO outputs from PyVista
logger.setLevel(logging.WARNING)

Plot the mean stress-strain curve and the axial stress field. Note that the stress field is now inhomogeneous.

In [ ]:
stress = material.extract("stress").mean("p").components[:, 2]
strain = material.extract("strain").mean("p").components[:, 2]
fig, ax = plt.subplots()
ax.plot(np.insert(strain, 0, 0), np.insert(stress, 0, 0))
ax.set_xlabel("axial strain")
ax.set_ylabel("axial stress [MPa]")

In [ ]:
material.plot("stress", component=2)

The slip system shear strains at each requested output time will now be added to `material.fields`, in addition to the stress and strain. The `Materialite` `Tensor` stored in the "slip_system_shear_strains" field will be a `Scalar` with dimensions "p", "s", and "t", representing the shear strain at each point, on each slip system, at each output time. Look at the slip system shear strains at the end of the simulation:

In [ ]:
slip_system_shear_strains = material.extract("slip_system_shear_strains")
print(f"Slip system shear strain dimensions: {slip_system_shear_strains.dims}")
print(slip_system_shear_strains[:, :, -1])

**Example 4: Uniaxial tension of a single crystal with a pore**

Here, we will simulate a plastically deforming matrix with a pore in the center. In this case, we need to assign a field to the material that identifies the phase (1 for the matrix and 2 for the pore).

In [ ]:
midpoint = 7.5
material = (
    Material(dimensions=[16, 16, 16])
    .create_uniform_field("orientation", Orientation.identity())
    .create_uniform_field("phase", 1)
    .insert_feature(
        Sphere(radius=3, centroid=[midpoint, midpoint, midpoint]),
        fields={"phase": 2},
    )
)
material.crop_by_range(x_range=(-np.inf, material.sizes[0] / 2)).plot("phase")

We will define separate constitutive models for the crystal and the pore. The pore will be an elastic model with zero stiffness. Materialite uses the method in refs. [3-4] to handle the infinite stiffness contrast between the pore and the solid material without convergence issues.

In [ ]:
stiffness_tensor = Order4SymmetricTensor.from_cubic_constants(
    C11=243300, C12=156700, C44=117800
)
evp_model = ElasticViscoplastic(
    stiffness=stiffness_tensor,
    slip_systems=SlipSystem.octahedral(),
    reference_slip_rate=1.0,
    rate_exponent=10.0,
    slip_resistance=143.0,
    hardening_function=voce,
    hardening_properties={"tau_1": 50.0, "theta_0": 1450.0, "theta_1": 95.0},
)
pore_model = Elastic(stiffness_tensor * 0.0)

The constitutive models are associated with the correct phases by creating a regional field in the `Material`. The constitutive model field must be named `constitutive_model`.

In [ ]:
phases = [1, 2]
models = [evp_model, pore_model]
regional_fields = pd.DataFrame({"phase": phases, "constitutive_model": models})
material = material.create_regional_fields(
    region_label="phase", regional_fields=regional_fields
)

Look at what we've added to the `Material`

In [ ]:
material.get_fields()

We will use the same `LoadSchedule` again. Note that no constitutive model is passed to `SmallStrainFFT`.

In [ ]:
load_schedule = LoadSchedule.from_constant_uniaxial_strain_rate(
    magnitude=1.0, direction="z"
)
model = SmallStrainFFT(
    load_schedule=load_schedule,
    end_time=0.007,
    initial_time_increment=0.001,
)

Run the model. We specify `phase_label="phase"` to tell the model to look at the `Material` field called `phase` to determine which points belong to which constitutive model. We will ask for the slip system shear strains as well.

In [ ]:
num_output_times = 7
output_times = np.linspace(start=0.001, stop=0.007, num=num_output_times)
material = material.run(
    model,
    phase_label="phase",
    output_times=output_times,
    output_variables=["slip_system_shear_strains"],
)

Compare the mean stress in the pore and solid material at each output time. As expected, the stress is zero at all points in the pore.

In [ ]:
indices = material.get_region_indices("phase")
pore_indices = indices[2]
solid_indices = indices[1]
stress = material.extract("stress")
pore_stress = stress[pore_indices]
solid_stress = stress[solid_indices]
mean_pore_stress = pore_stress.mean().stress_voigt
mean_solid_stress = solid_stress.mean().stress_voigt
print(f"mean pore stress: {mean_pore_stress}")
print(f"mean solid stress: {mean_solid_stress}")

Plot stress field of half the material to show stress near the pore

In [ ]:
material.crop_by_range(x_range=(-np.inf, material.sizes[0] / 2)).plot(
    "stress", component=2
)

Plot the mean stress-strain curve

In [ ]:
stress = material.extract("stress")
strain = material.extract("strain")
stress_solid = stress[solid_indices].mean("p").components[:, 2]
strain_solid = strain[solid_indices].mean("p").components[:, 2]
stress_pore = stress[pore_indices].mean("p").components[:, 2]
strain_pore = strain[pore_indices].mean("p").components[:, 2]
fig, ax = plt.subplots()
ax.plot(np.insert(strain_solid, 0, 0), np.insert(stress_solid, 0, 0), label="solid")
ax.plot(np.insert(strain_pore, 0, 0), np.insert(stress_pore, 0, 0), label="pore")
ax.legend()
ax.set_xlabel("Axial strain [-]")
ax.set_ylabel("Axial stress [MPa]")

Look at the slip system shear strains at the end of the simulation. Note that the values default to zero in the pore since this is not an available state variable in `Elastic` models.

In [ ]:
slip_system_shear_strains = material.extract("slip_system_shear_strains")
print(slip_system_shear_strains[solid_indices, :, -1])
print(slip_system_shear_strains[pore_indices, :, -1])

**Additional notes**

There are also `LoadSchedule` constructors for strain- and stress-based cyclic loading. Each one requires an amplitude, a frequency, and a stress mask.

The stress mask is a numpy array of length 6 that contains only 1s and 0s. 1s correspond to components where the mean stress is prescribed; 0s correspond to components where the mean strain is prescribed. The components are in the standard Voigt order (`[xx, yy, zz, yz, xz, xy]`)

In [ ]:
strain_amplitude = Order2SymmetricTensor.from_strain_voigt([0.01, 0, 0, 0, 0, 0])
stress_mask = np.array([0, 1, 1, 1, 1, 1])
load_schedule = LoadSchedule.from_cyclic_strain(
    strain_amplitude=strain_amplitude, frequency=1.0, stress_mask=stress_mask
)

In [ ]:
stress_amplitude = Order2SymmetricTensor.from_stress_voigt([0.01, 0, 0, 0, 0, 0])
stress_mask = np.ones(6)
load_schedule = LoadSchedule.from_cyclic_stress(
    stress_amplitude=stress_amplitude, frequency=1.0, stress_mask=stress_mask
)

You can also create your own `LoadSchedule`. You need to provide:
* `strain_increment`: a function that takes the previous time (`t`) and the current time increment (`dt`) as inputs and returns the mean strain increment as an `Order2SymmetricTensor` (i.e., `strain_increment(t, dt) -> Order2SymmetricTensor`)
* `stress_increment`: a function that takes `t` and `dt` as inputs and returns the mean stress increment as an `Order2SymmetricTensor`
* `stress`: a function that takes `t` and `dt` as inputs and returns the new mean stress as an `Order2SymmetricTensor`
* `stress_mask`: same as the definition for all of the `LoadSchedule`s above.

Create an instance of your load schedule: `load_schedule = LoadSchedule(strain_increment, stress_increment, stress, stress_mask)`

There are two other hardening laws that can be imported from `materialite.models.small_strain_fft`:

* `perfect_plasticity`: does not require any hardening law properties (i.e., `hardening_properties=None` in the constitutive law definition).
* `armstrong_frederick`: hardening law of the form $\dot{g}_s = \left(H - H_d g_s \right) \dot{\Gamma}$. The two required hardening properties are "direct_hardening" ($H$) and "dynamic_recovery" ($H_d$).

Other notes:
* When constitutive models are specified by phase (or another field in the `Material`), the model assumes there is a regional field associated with the phase called "constitutive_model".
* The model (`SmallStrainFFT`) increments the global stress and strain, uses conjugate gradients + FFTs [1,2] to estimate the global strain field given a guess global stress field, and asks the constitutive model for the updated global stress and consistent tangent fields.
* When phase-dependent constitutive models are specified, `SmallStrainFFT` uses a "utility" constitutive model class called `Multiphase` (see `materialite.models.small_strain_fft.multiphase`). `Multiphase` pulls off the material points corresponding to each constitutive model, calls the constitutive models, and reconstructs the global stress and consistent tangent fields from the output of each constitutive model.
* There are also `LoadSchedule` constructors for cyclic loading. 

[1] T.W.J. de Geus, J. Vondrejc, J. Zeman, R.H.J. Peerlings, M.G.D. Geers. Finite strain FFT-based non-linear solvers made simple. Computer Methods in Applied Mechanics and Engineering, 2017, 318:412–430. doi: 10.1016/j.cma.2016.12.032

[2] J. Zeman, T.W.J. de Geus, J. Vondrejc, R.H.J. Peerlings, M.G.D. Geers. A finite element perspective on nonlinear FFT-based micromechanical simulations. International Journal for Numerical Methods in Engineering, 2017, 111(10):903–926. doi: 10.1002/nme.5481

[3] F. Willot. Fourier-based schemes for computing the mechanical response of composites with accurate local fields. Comptes Rendus Mécanique, 2015, 343:232–245. doi: 10.1016/j.crme.2014.12.005

[4] S. Lucarini, L. Cobian, A. Voitus, J. Segurado. Adaptation and validation of FFT methods for homogenization of lattice based materials. Computer Methods in Applied Mechanics and Engineering, 2022, 388:114223. doi: 10.1016/j.cma.2021.114223